- Simular por semana, por dia o por hora???
- Si es por semana: Al inicio de cada semana sabemos que lotes compramos y sus tiempos de llegada
- Entonces, simulamos cada semana
- Input: información de los lotes comprados en esa semana=(calidad, tiempo de llegada)
- Definir clase "Camion"
- Las toneladas del lote comprado se dividen en camiones
- A cada camion de un mismo lote se le asigna un tiempo de llegada con separación de 1 minuto
- Diccionario de camiones: (lote, tipo de uva, toneladas, calidad, tiempo de llegada)

Etapas procesamiento:
- Molienda: 4 equipos cada uno con 3000 kg/hr y merma 0.05
- Prensado: 6 equipos cada uno con 2500 kg/hr y merma 0.15
- Clarificacion: 2 equipos cada uno con 5000 kg/hr y merma 0.35

**SIMULACION**

In [ ]:
import random
import math
from scipy.stats import poisson
import numpy as np

In [ ]:
#lotes comprados y que deben llegar dentro de la semana
#dicc_lotes_comprados={"loteX"=[tipo uva, toneladas, calidad x dia de cosecha, instante de llegada a fabrica]}
dicc_lotes_comprados={}
#creo el diccionario de camiones 
dicc_camiones={}
#asigno toneladas a los camiones
#para cada lote comprado
cap_max=3000
i=1
for lote in dicc_lotes_comprados.values():
    toneladas_lote=lote[1]
    TTOTALES+=toneladas_lote
    por_asignar=toneladas_lote
    #mientras queden toneladas del lote por asignar a un camion
    desfase=0
    while por_asignar>0:
        #creo el camion con su respectivo tipo de uva, toneladas, calidad actualizada x transporte, instante llegada.
        toneladas_camion=min(cap_max,por_asignar)
        dicc_camiones["Camion{}".format(i)]=[lote[0],toneladas_camion,lote[2],lote[3]+desfase]
        desfase+=1
        i+=1
        por_asignar-=toneladas_camion
#creo lista donde guardo los instantes de llegada de todos los camiones de menor a mayor
TLLEG=[]
for camion in dicc_camiones.values():
    TLLEG.append(camion[3])
TLLEG.sort()

In [ ]:
def sacar_o_bloquear1(M,TPE2M1,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,T,kg):
    if P1==0:
        M=0
        TPE2M1=math.inf
        P1=1
        TPE2P1=T+(kg/2500)
    elif P2==0:
        M=0
        TPE2M1=math.inf
        P2=1
        TPE2P2=T+(kg/2500)
    elif P3==0:
        M=0
        TPE2M1=math.inf
        P3=1
        TPE2P3=T+(kg/2500)
    elif P4==0:
        M=0
        TPE2M1=math.inf
        P4=1
        TPE2P4=T+(kg/2500)
    elif P5==0:
        M=0
        TPE2M1=math.inf
        P5=1
        TPE2P5=T+(kg/2500)
    elif P6==0:
        M=0
        TPE2M1=math.inf
        P6=1
        TPE2P6=T+(kg/2500)
    else:
        M=2  
    return M,TPE2M1,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6

In [ ]:
def ver_cola(cola,estado_equipo,tiempo_salida,T):
    if len(cola)==0: #si no hay camiones en cola
        estado_equipo=0 #actualizo el estado del equipo a desocupado
        tiempo_salida=math.inf #actualizo el tiempo de salida del equipo a infinito
        print("No hay camiones esperando en cola")
    else: #si hay camiones en cola
        estado_equipo=1 #actualizo el estado del equipo a ocupado
        tiempo_salida=T+(cola[0][2]/3000) #actualizo el tiempo de salida del equipo
        cola.pop(0) #saco al camion de la cola
        print("Entra el primer camion en cola")
    return estado_equipo,tiempo_salida,cola

In [ ]:
def sacar_o_bloquear2(P,TPE2P,C1,TPE2C1,C2,TPE2C2,T,kg):
    if C1==0:
        P=0
        TPE2P=math.inf
        C1=1
        TPE2C1=T+(kg/5000)
    elif C2==0:
        P=0
        TPE2P=math.inf
        C2=1
        TPE2C2=T+(kg/5000)
    else:
        P=2  
    return P,TPE2P,C1,TPE2C1,C2,TPE2C2

In [ ]:
def ver_bloqueo1(M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P,TPE2P,T,kg)
    if M1==2:
        M1=0
        TPE2M1=math.inf
        P=1
        TPE2P=T+(kg/2500)
    elif M2==2:
        M2=0
        TPE2M2=math.inf
        P=1
        TPE2P=T+(kg/2500)
    elif M3==2:
        M3=0
        TPE2M3=math.inf
        P=1
        TPE2P=T+(kg/2500)
    elif M4==2:
        M4=0
        TPE2M4=math.inf
        P=1
        TPE2P=T+(kg/2500)
    else:
        P=0
        TPE2P=math.inf
    return M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P,TPE2P

In [ ]:
def ver_bloqueo2(P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,C,TPE2C,T,kg)
    if P1==2:
        P1=0
        TPE2P1=math.inf
        C=1
        TPE2C=T+(kg/5000)
    elif P2==2:
        P2=0
        TPE2P2=math.inf
        C=1
        TPE2C=T+(kg/5000)
    elif P3==2:
        P3=0
        TPE2P3=math.inf
        C=1
        TPE2C=T+(kg/5000)
    elif P4==2:
        P4=0
        TPE2P4=math.inf
        C=1
        TPE2C=T+(kg/5000)
    elif P5==2:
        P5=0
        TPE2P5=math.inf
        C=1
        TPE2C=T+(kg/5000)
    elif P6==2:
        P6=0
        TPE2P6=math.inf
        C=1
        TPE2C=T+(kg/5000)
    else:
        C=0
        TPE2C=math.inf
    return P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,C,TPE2C

In [ ]:
##AHORA EMPIEZO LA SIMULACION##

T=0 #reloj simulacion
#TTOTALES=0 #suma de las toneladas de los camiones que llegan en el intervalo de tiempo
#TPROCESADAS=0 #toneladas que han sido procesadas

M1=0 #equipo 1 molienda desocupado
M2=0 #equipo 2 molienda desocupado
M3=0 #equipo 3 molienda desocupado
M4=0 #equipo 4 molienda desocupado
P1=0 #equipo 1 prensado desocupado
P2=0 #equipo 2 prensado desocupado
P3=0 #equipo 3 prensado desocupado
P4=0 #equipo 4 prensado desocupado
P5=0 #equipo 5 prensado desocupado
P6=0 #equipo 6 prensado desocupado
C1=0 #equipo 1 clarificacion desocupado
C2=0 #equipo 2 clarificacion desocupado

TPE1=TLLEG[0] #instante llegada del proximo camion
kg_camion=0
lote_camion=0

TPE2M1=math.inf #instante salida de equipo 1 molienda
TPE2M2=math.inf #instante salida de equipo 2 molienda
TPE2M3=math.inf #instante salida de equipo 3 molienda
TPE2M4=math.inf #instante salida de equipo 4 molienda
TPE2P1=math.inf #instante salida de equipo 1 prensado
TPE2P2=math.inf #instante salida de equipo 2 prensado
TPE2P3=math.inf #instante salida de equipo 3 prensado
TPE2P4=math.inf #instante salida de equipo 4 prensado
TPE2P5=math.inf #instante salida de equipo 5 prensado
TPE2P6=math.inf #instante salida de equipo 6 prensado
TPE2C1=math.inf #instante salida de equipo 1 clarificacion
TPE2C2=math.inf #instante salida de equipo 2 clarificacion


while T<0:
    prox_evento=min(TPE1,TPE2M1,TPE2M2,TPE2M3,TPE2M4,TPE2P1,TPE2P2,TPE2P3,TPE2P4,TPE2P5,TPE2P6,TPE2C1,TPE2C2)
    #si el prox evento es una llegada
    if prox_evento==TPE1:
        #actualizo reloj simulacion
        T=prox_evento
        #proxima llegada
        TPE1=math.inf
        #info del camion que llega
        lote_camion=0
        uva_camion=0
        kg_camion=0
        calidad=0 #calidad con la que fue cosechada
        tiempo_fuera=0
        print("-------->Llega un camion con {} kg del lote {} a las {} horas".format(kg_camion,lote_camion,T))
        #ver si algun equipo de molienda esta desocupado
        if M1==0:
            M1=1 #se actualiza estado del equipo a ocupado
            TPE2M1=T+(kg_camion/3000) #instante en que saldra esta carga de la lína
            print("Entra al equipo 1 de molienda, saldra a las {} horas".format(TPE2M1))
            #se actualiza el instante de llegada del prox camion
        elif M2==0:
            M2=1 #se actualiza estado del equipo a ocupado
            TPE2M2=T+(kg_camion/3000) #instante en que saldra esta carga de la lína
            print("Entra al equipo 2 de molienda, saldra a las {} horas".format(TPE2M2))
            #se actualiza el instante de llegada del prox camion
        elif M3==0:
            M3=1 #se actualiza estado del equipo a ocupado
            TPE2M3=T+(kg_camion/3000) #instante en que saldra esta carga de la lína
            print("Entra al equipo 3 de molienda, saldra a las {} horas".format(TPE2M3))
            #se actualiza el instante de llegada del prox camion
        elif M4==0:
            M4=1 #se actualiza estado del equipo a ocupado
            TPE2M4=T+(kg_camion/3000) #instante en que saldra esta carga de la lína
            print("Entra al equipo 1 de molienda, saldra a las {} horas".format(TPE2M4))
            #se actualiza el instante de llegada del prox camion
        else:
            #se incorpora a la cola
            COLA.append([lote_camion,uva_camion,kg_camion,calidad,tiempo_fuera])
            print("El camion se incorpora a la cola")
            
    #si el prox evento es una salida de molienda
    elif prox_evento==TPE2M1 or prox_evento==TPE2M2 or prox_evento==TPE2M3 or prox_evento==TPE2M4:
        #actualizo reloj simulacion
        T=prox_evento
        #Salida de equipo 1 molienda
        if prox_evento==TPE2M1:
            #Veo si entra a prensado o se bloquea
            M1,TPE2M1,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6=sacar_o_bloquear1(M1,TPE2M1,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,T,kg)
            #Si entra a prensado
            if M1==0:
                M1,TPE2M1,COLA=ver_cola(COLA,M1,TPE2M1,T)                                                                                               
        #Salida de equipo 2 molienda
        elif prox_evento==TPE2M2:
            #Veo si entra a prensado o se bloquea
            M2,TPE2M2,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6=sacar_o_bloquear1(M2,TPE2M2,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,T,kg)
            #Si entra a prensado
            if M2==0:
                M2,TPE2M2,COLA=ver_cola(COLA,M2,TPE2M2,T) 
        #Salida de equipo 3 molienda
        elif prox_evento==TPE2M3:
            #Veo si entra a prensado o se bloquea
            M3,TPE2M3,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6=sacar_o_bloquear1(M3,TPE2M3,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,T,kg)
            #Si entra a prensado
            if M3==0:
                M3,TPE2M3,COLA=ver_cola(COLA,M3,TPE2M3,T) 
        #Salida de equipo 4 molienda
        elif prox_evento==TPE2M4:
            #Veo si entra a prensado o se bloquea
            M4,TPE2M4,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6=sacar_o_bloquear1(M4,TPE2M4,P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,T,kg)
            #Si entra a prensado
            if M4==0:
                M4,TPE2M4,COLA=ver_cola(COLA,M4,TPE2M4,T) 
            
    #si el prox evento es una salida de prensado
    elif prox_evento==TPE2P1 or prox_evento==TPE2P2 or prox_evento==TPE2P3 or prox_evento==TPE2P4 or prox_evento==TPE2P5 or prox_evento==TPE2P6:
        #actualizo reloj
        T=prox_evento
        #Salida de equipo 1 prensado
        if prox_evento==TPE2P1:
            #Veo si entra a clarificacion o se bloquea
            P1,TPE2P1,C1,TPE2C1,C2,TPE2C2=sacar_o_bloquear2(P1,TPE2P1,C1,TPE2C1,C2,TPE2C2,T,kg)
            #Si entra a clarificacion
            if P1==0:
                M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P1,TPE2P1=ver_bloqueo1(M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P1,TPE2P1,T,kg)
        elif prox_evento==TPE2P2:
            #Veo si entra a clarificacion o se bloquea
            P2,TPE2P2,C1,TPE2C1,C2,TPE2C2=sacar_o_bloquear2(P2,TPE2P2,C1,TPE2C1,C2,TPE2C2,T,kg)
            #Si entra a clarificacion
            if P2==0:
                M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P2,TPE2P2=ver_bloqueo1(M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P2,TPE2P2,T,kg)
        elif prox_evento==TPE2P3:
            #Veo si entra a clarificacion o se bloquea
            P3,TPE2P3,C1,TPE2C1,C2,TPE2C2=sacar_o_bloquear2(P3,TPE2P3,C1,TPE2C1,C2,TPE2C2,T,kg)
            #Si entra a clarificacion
            if P3==0:
                M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P3,TPE2P3=ver_bloqueo1(M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P3,TPE2P3,T,kg)
        elif prox_evento==TPE2P4:
            #Veo si entra a clarificacion o se bloquea
            P4,TPE2P4,C1,TPE2C1,C2,TPE2C2=sacar_o_bloquear2(P4,TPE2P1,C4,TPE2C1,C2,TPE2C2,T,kg)
            #Si entra a clarificacion
            if P4==0:
                M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P4,TPE2P4=ver_bloqueo1(M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P4,TPE2P4,T,kg)
        elif prox_evento==TPE2P5:
            #Veo si entra a clarificacion o se bloquea
            P5,TPE2P5,C1,TPE2C1,C2,TPE2C2=sacar_o_bloquear2(P5,TPE2P5,C1,TPE2C1,C2,TPE2C2,T,kg)
            #Si entra a clarificacion
            if P5==0:
                M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P5,TPE2P5=ver_bloqueo1(M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P5,TPE2P5,T,kg)
        elif prox_evento==TPE2P6:
            #Veo si entra a clarificacion o se bloquea
            P6,TPE2P6,C1,TPE2C1,C2,TPE2C2=sacar_o_bloquear2(P6,TPE2P6,C1,TPE2C1,C2,TPE2C2,T,kg)
            #Si entra a clarificacion
            if P6==0:
                M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P6,TPE2P6=ver_bloqueo1(M1,TPE2M1,M2,TPE2M2,M3,TPE2M3,M4,TPE2M4,P6,TPE2P6,T,kg)
          
    #si el prox evento es una salida de clarificacion
    elif prox_evento==TPE2C1 or prox_evento==TPE2C2:
        #actualizo reloj
        T=prox_evento
        #Salida de equipo 1 clarificacion
        if prox_evento==TPE2C1:
            C1=0
            P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,C1,TPE2C1=ver_bloqueo2(P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,C2,TPE2C2,T,kg)
        #Salida de equipo 2 clarificacion
        elif prox_evento==TPE2C2:
            C2=0
            P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,C2,TPE2C2=ver_bloqueo2(P1,TPE2P1,P2,TPE2P2,P3,TPE2P3,P4,TPE2P4,P5,TPE2P5,P6,TPE2P6,C2,TPE2C2,T,kg)